### ECE475 Freq Machine Learning Recommendation System Assignment
Jing Jiang

I used MovieLens 100K dataset (MovieLens Small) in this assignment <br>
See https://grouplens.org/datasets/movielens/

In [12]:
import numpy as np
import pandas as pd

import os

r = pd.read_csv("./ml-latest-small/ratings.csv")
m = pd.read_csv("./ml-latest-small/movies.csv")

In [13]:
r.shape

(100836, 4)

In [14]:
m.shape

(9742, 3)

In [15]:
x = r.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
x.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X = x.as_matrix()
rating_mean = np.mean(X, axis = 1)
X_demean = X - rating_mean.reshape(-1, 1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


#### Matrix Factorization and  Singular Value Decomposition (SVD)

In [17]:
# SVD
from scipy.sparse.linalg import svds
u, sigma, vt = svds(X_demean, k = 50)
sigma = np.diag(sigma)

In [18]:
pred_rating = np.dot(np.dot(u, sigma), vt) + rating_mean.reshape(-1, 1)
preds = pd.DataFrame(pred_rating, columns = x.columns)
preds
# The matrix are shown at below. Each cell contains a prediction score showing whether a user might like a movie.
# Row is userID -1. Column is movieID.

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,2.167328,0.402751,0.840184,-0.076281,-0.551337,2.504091,-0.890114,-0.026443,0.196974,1.593259,...,-0.023453,-0.019967,-0.026939,-0.026939,-0.023453,-0.026939,-0.023453,-0.023453,-0.023453,-0.058732
1,0.211459,0.006658,0.033455,0.017419,0.183430,-0.062473,0.083037,0.024158,0.049330,-0.152530,...,0.019498,0.016777,0.022219,0.022219,0.019498,0.022219,0.019498,0.019498,0.019498,0.032281
2,0.003588,0.030518,0.046393,0.008176,-0.006247,0.107328,-0.012416,0.003779,0.007297,-0.059362,...,0.005909,0.006209,0.005610,0.005610,0.005909,0.005610,0.005909,0.005909,0.005909,0.008004
3,2.051549,-0.387104,-0.252199,0.087562,0.130465,0.270210,0.477835,0.040313,0.025858,-0.017365,...,0.004836,0.004172,0.005500,0.005500,0.004836,0.005500,0.004836,0.004836,0.004836,-0.023311
4,1.344738,0.778511,0.065749,0.111744,0.273144,0.584426,0.254930,0.128788,-0.085541,1.023455,...,-0.008042,-0.007419,-0.008664,-0.008664,-0.008042,-0.008664,-0.008042,-0.008042,-0.008042,-0.010127
5,4.256848,3.817145,3.191021,0.658325,3.329383,3.737443,3.602541,0.808500,1.288860,3.065296,...,0.061060,0.059230,0.062890,0.062890,0.061060,0.062890,0.061060,0.061060,0.061060,0.072276
6,2.193760,0.864982,-0.055281,-0.060124,0.063232,0.731992,0.011179,-0.105909,0.003810,1.459960,...,-0.022985,-0.021017,-0.024954,-0.024954,-0.022985,-0.024954,-0.022985,-0.022985,-0.022985,-0.019677
7,1.110016,1.145879,0.290815,0.117453,0.208646,0.333190,0.310963,0.090226,-0.174325,1.588648,...,-0.007117,-0.006463,-0.007771,-0.007771,-0.007117,-0.007771,-0.007117,-0.007117,-0.007117,-0.002879
8,0.095799,0.277416,0.056466,0.036945,0.049458,-0.196148,0.007290,0.004881,-0.034851,0.045509,...,0.003209,0.002924,0.003494,0.003494,0.003209,0.003494,0.003209,0.003209,0.003209,0.001469
9,0.659316,0.095336,-0.220904,-0.025737,0.244797,-0.487500,0.102795,-0.036848,-0.062763,0.399329,...,0.020021,0.020167,0.019876,0.019876,0.020021,0.019876,0.020021,0.020021,0.020021,0.050003


#### Recommendation Function

In [19]:
# Get and construct the user's rating data
def getRated(user, movies, ratings):
    user_data = ratings[ratings.userId == (user)]
    user_info = (user_data.merge(movies, how = 'left', left_on = 'movieId', 
                                 right_on = 'movieId').sort_values(['rating'], ascending=False))
    return user_info

In [20]:
# Predict the movies that the user may like in the list of movies he hasn't watched.
def recommend(pred, user_info, movies, num_recommendations=5):
    urow = user_info['userId'][0] - 1
    sorted_pred = pred.iloc[urow].sort_values(ascending=False)
    
    recommendations = (movies[~movies['movieId'].isin(user_info['movieId'])].
                       merge(pd.DataFrame(sorted_pred).reset_index(), how = 'left',
                              left_on = 'movieId', right_on = 'movieId').
                       rename(columns = {urow: 'predictions'}).
                       sort_values('predictions', ascending = False).
                       iloc[:num_recommendations, :-1])

    return recommendations

#### Recommendation Example
Let's say we want to make recommendations to User with ID number 2

In [21]:
user2 = getRated(2, m, r)
user2
# This is the list of movies user2 has rated. He has rated 29 movies.

,userId,movieId,rating,timestamp,title,genres
28,2,131724,5.0,1445714851,The Jinx: The Life and Deaths of Robert Durst ...,Documentary
27,2,122882,5.0,1445715272,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
22,2,106782,5.0,1445714966,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama
18,2,89774,5.0,1445715189,Warrior (2011),Drama
9,2,60756,5.0,1445714980,Step Brothers (2008),Comedy
16,2,80906,5.0,1445715172,Inside Job (2010),Documentary
2,2,1704,4.5,1445715228,Good Will Hunting (1997),Drama|Romance
8,2,58559,4.5,1445715141,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
10,2,68157,4.5,1445715154,Inglourious Basterds (2009),Action|Drama|War
15,2,80489,4.5,1445715340,"Town, The (2010)",Crime|Drama|Thriller


This guy actually has a good taste of movie. Mad Max Fury Road indeed deserves a 5.0

In [22]:
user2_pred = recommend(preds, user2, m, 15)
user2_pred
# We made 15 recommendations for User2 that he hasn't rated before. The movies are listed below.

,movieId,title,genres
2223,2959,Fight Club (1999),Action|Crime|Drama|Thriller
1936,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
7398,80463,"Social Network, The (2010)",Drama
312,356,Forrest Gump (1994),Comedy|Drama|Romance|War
8850,134130,The Martian (2015),Adventure|Drama|Sci-Fi
508,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
6323,48780,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
3634,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
7032,69122,"Hangover, The (2009)",Comedy|Crime
4795,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
